### Process Caucasus DZ Data ###
This notebook uses the accompanying dz module (dz.py) to process compiled Caucasus DZ data.

In [1]:
# Imports - run first
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geoscripts.dz import dz

This cell reads in the data from the Metadata table and creates DZ sample objects

In [2]:
# Import Data and Create DZ Objects
metadata = pd.read_excel('Metadata-Table-Caucasus.xlsx')

names = metadata['Sample Name']
sources = metadata['Source']
latitudes = metadata['Latitude (°N)']
longitudes = metadata['Longitude (°E)']
rep_ages = metadata['Reported Age']
sample_type = metadata['Type']

color_dict = {'Modern':'red','Bedrock':'blue'}

samples = [
    dz.DZSample(name,source=sources[x],latlon=(latitudes[x],longitudes[x]),
                reported_age=rep_ages[x],color=color_dict[sample_type[x]]) 
    for x,name in enumerate(names)
    ]

This cell loads in the age data from each samples' individual Excel file and adds it to the DZ object. It also plots KDEs for each sample

In [3]:
#%% Populate Objects with Age Data

paths = [sources[x] + '/' + name + '.xlsx' for x,name in enumerate(names)]

for x,sample in enumerate(samples):
    sample.agedata = pd.read_excel(paths[x])
    
    if '206/207 Pb Age' not in sample.agedata:
        sample.agedata['206/207 Pb Age'] = np.nan
    
    sample.calc_bestage('238/206 U-Pb Age','206/207 Pb Age')
    
    sample.kde_img()
    plt.close()
    
    sample.export_ages()
    sample.save()

This cell saves the data in the DZ objects to a SHP file for GIS

In [4]:
#%% Save to Shapefile

gdf = dz.write_file(samples,'caucasus_dz.shp')

c:\Users\dyvas\miniconda3\envs\dz\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\dyvas\git\dz-caucasus\geoscripts\dz\dz.py:650: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename,crs='EPSG:4326')


This cell exports modern samples to a separate csv for catchment analysis.

In [5]:
names = []
lats = []
lons = []
for sample in samples:
    if sample.color=='red':
        names.append(sample.name)
        lats.append(sample.latlon[0])
        lons.append(sample.latlon[1])

data = {'Modern Sample':names,'Latitude':lats,'Longitude':lons}
df = pd.DataFrame(data)
df.to_csv('modern_samples.csv')